## Putting It All Together

现在我们已经介绍了Pipeline API的基本组件，让我们构建一个我们可能想要在算法中使用的管道。
首先，我们首先创建一个过滤器来缩小我们管道中出现的证券类型。 在这个例子中，我们将创建一个过滤器来选择符合以下标准的证券：
+ 是主要的股票
+ 被列为普通股
+ 不是存托凭证（ADR / GDR）
+ 不在场外交易（OTC）
+ 不是 when-issued (WI) 
+ 没有名称表明它是有限合伙企业（LP）
+ 不是ETF（有晨星基本面数据）

### Why These Criteria?

选择主要股票和普通股有助于我们为每家公司选择一种单一的证券。 一般来说，主要股份是公司的一个很好的代表资产，所以我们将在我们的管道中选择这些资产。

ADR和GDR是美国股票市场上发行的在其他交易所交易的股票。 由于货币波动，存托凭证通常存在固有的风险，所以我们将其排除在我们的管道之外。
OTC，WI和LP证券不能与大多数经纪商进行交易。 因此，我们将其排除在我们的管道之外。

在对证券进行排名和比较时，将ETF与常规股票进行比较是很有意义的。 ETF是没有基本数据的复合材料。 他们从一大群证券中获得价值。 为了避免比较苹果和橙子，我们将它们排除在我们的管道之外。

## Creating Our Pipeline

我们为每个标准创建一个过滤器，并将它们组合在一起以创建一个tradeable_stocks过滤器。 首先，我们需要导入Morningstar DataSet以及IsPrimaryShare内置过滤器。

请注意，在定义我们的过滤器时，我们使用了几个尚未见过的`Classifier`方法，包括`notnull`，`startswith`，`endswith`和`matches`。这些方法的文档可以在[这里](https://www.quantopian.com/help#quantopian_pipeline_classifiers_Classifier)找到。

In [1]:
from zipline.pipeline.fundamentals import Fundamentals
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import AverageDollarVolume, SimpleMovingAverage
from zipline.pipeline import Pipeline
from zipline.research import run_pipeline

In [2]:
# 下列用处不大，主要是熟悉用法
not_st = ~Fundamentals.short_name.latest.has_substring('ST')
normal_stock = ~Fundamentals.treatment.latest.element_of(['new','ST','delisting'])
have_market_cap = USEquityPricing.cmv.latest.notnull()
tradeable_stocks = (
    not_st
    & normal_stock
    & have_market_cap
)

接下来，让我们以20天的平均美元成交量为前30％的可交易股票创建一个过滤器。 我们将这称为我们的base_universe。

In [3]:
# 放大到60-100
base_universe = AverageDollarVolume(window_length=20, mask=tradeable_stocks).percentile_between(60, 100)

### Mean Reversion Factors

现在我们有一个可用于选择证券子集的过滤器`base_universe`，让我们专注于为这个子集创建因子。对于这个例子，我们来创建一个平均反转策略的流水线。 这个策略中，我们将观察10天和30天移动平均线（收盘价）。我们计划在最小百分比差异的75个证券下达等权多头头寸，而百分比差异最大75个证券下达等权空头头寸。 为此，使用`base_universe`过滤器作为掩码创建两个移动平均因子。然后我们将它们组合成一个计算百分比差异的因子。

In [4]:
def make_pipeline():

    # Base universe filter.
    base_universe = AverageDollarVolume(window_length=20, mask=tradeable_stocks).percentile_between(60, 100)

    # 10-day close price average.
    mean_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=10, mask=base_universe)

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=30, mask=base_universe)

    # Percent difference factor.
    percent_difference = (mean_10 - mean_30) / mean_30

    # Create a filter to select securities to short.
    shorts = percent_difference.top(75)

    # Create a filter to select securities to long.
    longs = percent_difference.bottom(75)

    # Filter for the securities that we want to trade.
    securities_to_trade = (shorts | longs)

    return Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts
        }, screen=securities_to_trade)

运行此管道将导致包含2列的DataFrame。 每天，这些列将包含布尔值，我们可以使用这些值来决定是否要在每个证券中打开多头头寸或空头头寸。

In [5]:
result = run_pipeline(make_pipeline(), '2018-04-05', '2018-04-15')
result.head()

/home/ldf/source/czipline/zipline/pipeline/filters/filter.py:380: RuntimeWarning: invalid value encountered in less_equal
  return (lower_bounds <= data) & (data <= upper_bounds)


longs  shorts
2018-04-10 00:00:00+00:00 全新好(000007)    True   False
                          神州高铁(000008)   True   False
                          沙河股份(000014)  False    True
                          神州长城(000018)  False    True
                          特 力Ａ(000025)   True   False

在下一课中，我们将把这个管道添加到算法中。